In [1]:
from database import *

def ticks_to_kbars(ticks, interval='5Min'):

    kbars = pd.DataFrame()
    
    kbars['open'] = ticks['close'].resample(interval).first()
    kbars['close'] = ticks['close'].resample(interval).last()
    kbars['high'] = ticks['close'].resample(interval).max()
    kbars['low'] = ticks['close'].resample(interval).min()
    kbars['volume'] = ticks['volume'].resample(interval).sum()
    
    kbars.dropna(inplace=True)
    
    return kbars

In [2]:
import talib

def get_technical_indicator(kbars):
    
#     kbars['rsi'] = talib.RSI(kbars.close, timeperiod=14)
    kbars['cci'] = talib.CCI(kbars.high, kbars.low, kbars.close, timeperiod=14)
    
    macd, macdsignal, macdhist = talib.MACD(kbars.close, fastperiod=5, slowperiod=34, signalperiod=5)
    
    kbars['macd'] = macd
    kbars['macdsignal'] = macdsignal
    kbars['macdhist'] = macdhist
    
    kbars['slowk'], kbars['slowd'] = talib.STOCH(kbars.high,
                        kbars.low,
                        kbars.close,
                        fastk_period=9,
                        slowk_period=3,
                        slowk_matype=0,
                        slowd_period=3,
                        slowd_matype=0)
    
    return kbars

In [3]:
import json

with open('config.json') as f:
  config = json.load(f)

api = sj.Shioaji()
api.login(
    person_id = config['account'],
    passwd= config['password'],
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done."))

# api.logout()

connection = sqlite3.connect('data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done.


In [36]:
code = '2399'

tw_calendar = get_calendar('XTAI')

date = pd.to_datetime('2020-2-3')
prev_trading_date = tw_calendar.previous_close(date).date()

ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
kbars = ticks_to_kbars(ticks)
kbars = get_technical_indicator(kbars)
# kbars = kbars[pd.to_datetime(prev_trading_date).replace(hour=13, minute=30):]
kbars = kbars[date:]

kbars

,open,close,high,low,volume,cci,macd,macdsignal,macdhist,slowk,slowd
ts,,,,,,,,,,,
2020-02-03 09:00:00,10.30,9.90,10.35,9.90,203,-421.232123,-0.229540,-0.057456,-0.172084,-9.473903e-15,11.111111
2020-02-03 09:05:00,9.90,9.90,9.96,9.86,92,-258.434596,-0.364963,-0.159959,-0.205004,1.418440e+00,0.472813
2020-02-03 09:10:00,9.90,9.86,9.91,9.86,88,-172.088778,-0.454182,-0.258033,-0.196149,1.418440e+00,0.945626
2020-02-03 09:15:00,9.86,9.87,9.87,9.86,75,-128.122912,-0.498849,-0.338305,-0.160544,1.773050e+00,1.536643
2020-02-03 09:20:00,9.87,9.88,9.88,9.86,52,-100.454133,-0.514662,-0.397091,-0.117571,1.063830e+00,1.418440
2020-02-03 09:25:00,9.88,9.85,9.88,9.85,78,-85.048860,-0.523084,-0.439088,-0.083996,1.063830e+00,1.300236
2020-02-03 09:30:00,9.88,9.91,9.92,9.88,83,-66.966461,-0.501843,-0.460007,-0.041836,2.931442e+00,1.686367
2020-02-03 09:35:00,9.91,9.88,9.91,9.88,24,-60.490973,-0.487218,-0.469077,-0.018141,3.472222e+00,2.489165
2020-02-03 09:40:00,9.95,9.93,10.00,9.93,56,-44.025157,-0.454936,-0.464363,0.009428,8.805556e+00,5.069740


In [5]:
def day_trading_backtest(code, date, connection, api):
    
    tw_calendar = get_calendar('XTAI')
    prev_trading_date = tw_calendar.previous_close(date).date()
    
    ticks = get_ticks(code, prev_trading_date, connection, api)[0].append(get_ticks(code, date, connection, api)[0])
    kbars = ticks_to_kbars(ticks)
    kbars = get_technical_indicator(kbars)
    kbars = kbars[date:]
            
    entry_price = 0
    entry_time = None
    
    exit_price = 0
    exit_time = None
    
    position = 0
            
    for ts in range(len(kbars)):
        
        current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=5)
        print(' start current_time = {}'.format(current_time))
        current_price = kbars['close'][ts]
        print('current_price = {}'.format(current_price))
#         macdhist = kbars.iloc[ts]['macdhist']
#         print('macdhist = {}'.format(macdhist))
#         cci = kbars.iloc[ts]['cci']
#         print('cci = {}'.format(cci))
#         print('end')
        
        if (
#             kbars.iloc[ts - 3:ts]['rsi'].min() < 30 and
#             kbars.iloc[ts - 1]['macdhist'] < 0 and
            current_time < date.replace(hour=13, minute=20, second=0) and
            kbars.iloc[ts]['cci'] > -100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position == 0
        ):
            
            position = 1          
            entry_price = current_price
            entry_time = current_time.time()

            print('[{}] buy {} at {}'.format(current_time, code, current_price))
        
        elif (
#             kbars.iloc[ts - 3:ts]['rsi'].max() > 70 and
#             kbars.iloc[ts - 1]['macdhist'] > 0 and
            kbars.iloc[ts]['cci'] > 100 and
            kbars.iloc[ts]['macdhist'] > 0 and
            position != 0
        ):
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
        elif (
            current_time >= date.replace(hour=13, minute=20, second=0) and
            position != 0
        ):
            
            exit_price = current_price
            exit_time = current_time.time()
            
            print('[{}] sell {} at {}'.format(current_time, code, current_price))
            
            break
            
    if entry_time and exit_time:
        transaction = pd.DataFrame([[date, 
                                     code, 
                                     entry_time,
                                     entry_price, 
                                     position * 1000,
                                     entry_price * position * 1000,
                                     exit_time,
                                     exit_price,
                                     position * 1000,
                                     exit_price * position * 1000]
                                   ],
                                    columns=[
                                        '成交日期', 
                                        '股票代號',
                                        '買進時間',
                                        '買進價格',
                                        '買進股數',
                                        '買進金額',
                                        '賣出時間',
                                        '賣出價格',
                                        '賣出股數',
                                        '賣出金額'])
        return transaction
    else:
        return pd.DataFrame()

In [6]:
day_trading_backtest('2456', pd.to_datetime('2021/3/2'), connection, api)

 start current_time = 2021-03-02 09:05:00
current_price = 114.0
[2021-03-02 09:05:00] buy 2456 at 114.0
 start current_time = 2021-03-02 09:10:00
current_price = 112.5
[2021-03-02 09:10:00] sell 2456 at 112.5


,成交日期,股票代號,買進時間,買進價格,買進股數,買進金額,賣出時間,賣出價格,賣出股數,賣出金額
0,2021-03-02,2456,09:05:00,114.0,1000,114000.0,09:10:00,112.5,1000,112500.0
